In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import math
import time
import sys
import os
import random
import pickle

import numpy as np

import tensorflow.keras
import tensorflow.keras.layers
import tensorflow.keras.applications
import tensorflow.keras.backend
import tensorflow.keras.preprocessing.image
import tensorflow.keras.utils
import tensorflow as tf

import cv2
import PIL
import PIL.Image
import matplotlib.pyplot as plt

import shutil

from tensorflow.keras import backend as K


# configurations

## seeding
os.environ['PYTHONHASHSEED'] = '3'
np.random.seed(3)
random.seed(3)
tf.random.set_seed(3)

## which gpu to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## memory allocation
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.compat.v1.Session(config=config)
#K.set_session(session)
tf.compat.v1.keras.backend.set_session(session)
## data directory for CUB200 root
PATH_DATA_ROOT_CUB200 = "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/CUB_200_2011/CUB_200_2011"

## network configurations
### number of output classes, 200 for CUB200
NO_CLASS = 200



In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/CUB_200_2011/CUB_200_2011"

In [ ]:
from tensorflow.keras.initializers import glorot_normal
from keras.layers import Lambda

def outer_product(x):
    """
    calculate outer-products of 2 tensors

        args 
            x
                list of 2 tensors
                , assuming each of which has shape = (size_minibatch, total_pixels, size_filter)
    """
    return tensorflow.keras.backend.batch_dot(
                x[0]
                , x[1]
                , axes=[1,1]
            ) / x[0].get_shape().as_list()[1] 

def signed_sqrt(x):
    """
    calculate element-wise signed square root

        args
            x
                a tensor
    """
    return tensorflow.keras.backend.sign(x) * tensorflow.keras.backend.sqrt(tensorflow.keras.backend.abs(x) + 1e-9)

def L2_norm(x, axis=-1):
    """
    calculate L2-norm

        args 
            x
                a tensor
    """
    return tensorflow.keras.backend.l2_normalize(x, axis=axis)


def build_model(filter_zero##added aprameter
                ,value_filter
    ,size_heigth=448
    ,size_width=448
    ,no_class=200
    ,no_last_layer_backbone=17
    
    ,name_optimizer="sgd"
    ,rate_learning=1.0
    ,rate_decay_learning=0.0
    ,rate_decay_weight=0.0
    
    ,name_initializer="glorot_normal"
    ,name_activation_logits="softmax"
    ,name_loss="categorical_crossentropy"
    ,flg_debug=False
    ,**kwargs
):
    
    tensorflow.keras.backend.clear_session()
    
    # print("-------------------------------")
    # print("parameters:")
    # for key, val in locals().items():
    #     if not val == None and not key == "kwargs":
    #         print("\t", key, "=",  val)
    # print("-------------------------------")
    
    ### 
    ### load pre-trained model
    ###
    tensor_input = tensorflow.keras.layers.Input(shape=[size_heigth,size_width,3])
    model_detector = tensorflow.keras.applications.vgg16.VGG16(
                            input_tensor=tensor_input
                            , include_top=False
                            , weights='imagenet'
                        )
    
    arryWeights_last_After =[]
    for i in range(512):
      arryWeights_last_After.append(i)
    for i in range(512):
      arryWeights_last_After[i]=1
    
    arryWeights_last_After[filter_zero]=value_filter
    # print('filter' , filter_zero, "=", arryWeights_last_After[filter_zero])
    def custom_layer_last(tensor):
        return tensor * arryWeights_last_After
    lambda_layer = Lambda (custom_layer_last, name="lambda_New")

    model_detector = insert_intermediate_layer_in_keras (model_detector, 18, lambda_layer)
    # print('model detector')
    # model_detector.summary()
    ### 
    ### bi-linear pooling
    ###

    # extract features from detector
    x_detector = model_detector.layers[no_last_layer_backbone].output
    shape_detector = model_detector.layers[no_last_layer_backbone].output_shape
    # if flg_debug:
    #     print("shape_detector : {}".format(shape_detector))

    # extract features from extractor , same with detector for symmetry DxD model
    shape_extractor = shape_detector
    x_extractor = x_detector
    # if flg_debug:
    #     print("shape_extractor : {}".format(shape_extractor))
        
    
    # rehape to (minibatch_size, total_pixels, filter_size)
    x_detector = tensorflow.keras.layers.Reshape(
            [
                shape_detector[1] * shape_detector[2] , shape_detector[-1]
            ]
        )(x_detector)
    # if flg_debug:
    #     print("x_detector shape after rehsape ops : {}".format(x_detector.shape))
        
    x_extractor = tensorflow.keras.layers.Reshape(
            [
                shape_extractor[1] * shape_extractor[2] , shape_extractor[-1]
            ]
        )(x_extractor)
    # if flg_debug:
    #     print("x_extractor shape after rehsape ops : {}".format(x_extractor.shape))
        
        
    # outer products of features, output shape=(minibatch_size, filter_size_detector*filter_size_extractor)
    x = tensorflow.keras.layers.Lambda(outer_product)(
        [x_detector, x_extractor]
    )
    # if flg_debug:
    #     print("x shape after outer products ops : {}".format(x.shape))
        
        
    # rehape to (minibatch_size, filter_size_detector*filter_size_extractor)
    x = tensorflow.keras.layers.Reshape([shape_detector[-1]*shape_extractor[-1]])(x)
    # if flg_debug:
    #     print("x shape after rehsape ops : {}".format(x.shape))
        
        
    # signed square-root 
    x = tensorflow.keras.layers.Lambda(signed_sqrt)(x)
    # if flg_debug:
    #     print("x shape after signed-square-root ops : {}".format(x.shape))
        
    # L2 normalization
    x = tensorflow.keras.layers.Lambda(L2_norm)(x)
    # if flg_debug:
    #     print("x shape after L2-Normalization ops : {}".format(x.shape))



    ### 
    ### attach FC-Layer
    ###

    if name_initializer != None:
            name_initializer = eval(name_initializer+"()")
            
    x = tensorflow.keras.layers.Dense(
            units=no_class
            ,kernel_regularizer=tensorflow.keras.regularizers.l2(rate_decay_weight)
            ,kernel_initializer=name_initializer
        )(x)
    # if flg_debug:
    #     print("x shape after Dense ops : {}".format(x.shape))
    tensor_prediction = tensorflow.keras.layers.Activation(name_activation_logits)(x)
    # if flg_debug:
    #     print("prediction shape : {}".format(tensor_prediction.shape))

        

    ### 
    ### compile model
    ###
    model_bilinear = tensorflow.keras.models.Model(
                        inputs=[tensor_input]
                        , outputs=[tensor_prediction]
                    )
    
    
    # fix pre-trained weights
    for layer in model_detector.layers:
        layer.trainable = False
        
        
    # define optimizers
    opt_adam = tensorflow.keras.optimizers.Adam(
                    lr=rate_learning
                    , decay=rate_decay_learning
                )
    opt_rms = tensorflow.keras.optimizers.RMSprop(
                    lr=rate_learning
                    , decay=rate_decay_learning
                )
    opt_sgd = tensorflow.keras.optimizers.SGD(
                    lr=rate_learning
                    , decay=rate_decay_learning
                    , momentum=0.9
                    , nesterov=False
                )
    optimizers ={
        "adam":opt_adam
        ,"rmsprop":opt_rms
        ,"sgd":opt_sgd
    }
    
    model_bilinear.compile(
        loss=name_loss
        , optimizer=optimizers[name_optimizer]
        , metrics=["categorical_accuracy"]
    )
    
    
    
    # if flg_debug:
    #     model_bilinear.summary()
    
    return model_bilinear

In [ ]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    model = Model(inputs=layers[0].input, outputs=x) ### inputs instead of input in this version
    return model

In [ ]:
model = build_model(filter_zero=0
                    , value_filter=1
            # number of output classes, 200 for CUB200
            ,no_class = NO_CLASS

            # pretrained model specification, using VGG16
            # "block5_conv3 "
            ,no_last_layer_backbone = 18

            # training parametes
            ,rate_learning=1.0
            ,rate_decay_weight=1e-8
            ,flg_debug=True
        )

# model = build_model(
#             # number of output classes, 200 for CUB200
#             no_class = NO_CLASS

#             # pretrained model specification, using VGG16
#             # "block5_conv3 "
#             ,no_last_layer_backbone = 17           
#             # training parametes
#             ,rate_learning=1.0
#             ,rate_decay_weight=1e-8
#             ,flg_debug=True
#         )

In [ ]:
# now all layers are trainable
for layer in model.layers:
    layer.trainable = True

# change LR
opt_sgd = tensorflow.keras.optimizers.SGD(
                lr=1e-3
                , decay=1e-9
                , momentum=0.9
                , nesterov=False
            )
model.compile(
    loss="categorical_crossentropy"
    , optimizer=opt_sgd
    , metrics=["categorical_accuracy"]
)

In [ ]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/birds_200_weights.h5")

In [ ]:
import os

birds_labels = set()

path= "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/CUB_200_2011/CUB_200_2011/images"
for d in os.listdir(path):
    birds_labels.add(d)

len(birds_labels)

# print(birds_labels)


In [ ]:
birds_labels = list(birds_labels)
birds_labels.sort()
# birds_labels

In [ ]:
import cv2
def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (448,448))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

def output(location, new_model):
    img = load_and_preprocess_image(location)
    img = np.expand_dims(img, axis=0)
    answer = new_model.predict(img)
    y_class = answer.argmax(axis = -1)
    
    top_3 = np.argsort(answer[0])[:-4:-1]
    # for i in range(3):
    #     print(" ({:.3})".format(answer[0][top_3[i]]))
    
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = birds_labels[y]
    # print(res)
#     print(" ({:.3})".format(answer[0][top_3[0]]))
    pred_prob = answer[0][top_3[0]]
    return float(pred_prob), res

In [ ]:
location = "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/Birds_classes_top/011.Rusty_Blackbird/Rusty_Blackbird_0091_6695.jpg"
output(location, model)

# Choose top prototypes for causal learning process

In [ ]:
# import cv2
# import os
# from shutil import copyfile
# from pathlib import Path

# folder = "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/CUB_200_2011/CUB_200_2011/images/"
# folder_Copy = "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/Birds_classes_top/"

# last_fileName=""

# #images = []
# imagesFileNames = []
# for filename in os.listdir(folder):
#     if filename not in os.listdir(folder_Copy):
#         for img in os.listdir(folder+filename):
#             if output(folder+filename+"/"+img, model)[0] >= 0.9 :
#                 if output(folder+filename+"/"+img, model)[1] ==  filename:
#                     print('path', folder+filename+"/"+img)
#                     print('prob=', output(folder+filename+"/"+img, model)[0])
#                     src = folder+filename+"/"+img
#                     dst = folder_Copy+filename+"/"+img
#                     Path(folder_Copy+filename+"/").mkdir(parents=True, exist_ok=True)
#                     copyfile(src, dst)
#                     print(filename)
#         #             print(img)
#                     break;
#     #     img = cv2.imread(os.path.join(folder,filename))
#     #     if img is not None:
#     #         #images.append(img)
#         imagesFileNames.append(filename)
# imagesFileNames.sort()        
# print(imagesFileNames[0])
# #images = load_images_from_folder(folder)


In [ ]:
import cv2
import os
from shutil import copyfile
from pathlib import Path


folder = "/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/Birds_classes_top/"

last_fileName=""
#images = []
imagesFileNames = []
for filename in os.listdir(folder):
    # for img in os.listdir(folder+filename):
    imagesFileNames.append(filename)
    print(filename)
# imagesFileNames.sort()
print(len(imagesFileNames))

In [ ]:
# arryWeights_last_After =[]
# for i in range(512):
#   arryWeights_last_After.append(i)
# for i in range(512):
#   arryWeights_last_After[i]=1

# def custom_layer_last(tensor):
#     return tensor * arryWeights_last_After[i]
# # vgg_weights = VGG16(    input_shape = IMAGE_SIZE + [3], weights = 'imagenet',
# #     include_top = False).get_weights()

# This part for computing the CI and storing it into Dictionnary in tx file

In [ ]:
from keras.layers import Lambda
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, MaxPool2D 


weights_dic = {}

imgCount = 0
indexImg=0
# img_path = 'Mountain_Bike/[image-net.org][27]443691662_09f12b8e37.jpg'
for imgFileName in imagesFileNames:
    dctOfAllDictionClasses = {}
    imgPath = folder + imagesFileNames[indexImg] + "/" + os.listdir(folder + imagesFileNames[indexImg])[0]
    print(imgPath)
    res = output(imgPath, model)

    originalName = imgFileName
    print('orign name:', originalName)
    class_Id = res[1]
    orig_acc = res[0]
    print('origin acc', orig_acc)
    from keras.layers import Lambda
    weights_dic = {}
    K.set_learning_phase(0)
    
    import time
    start_time = time.time()
    for i in range(512):
        print ('filt{}'.format (i))
        filt = i

        new_model = build_model( filter_zero=i##added aprameter
            ,value_filter =0
            # number of output classes, 200 for CUB200
            ,no_class = NO_CLASS

            # pretrained model specification, using VGG16
            # "block5_conv3 "
            ,no_last_layer_backbone = 18
                        # training parametes
            ,rate_learning=1.0
            ,rate_decay_weight=1e-8
            ,flg_debug=True
        )

                # now all layers are trainable
        for layer in new_model.layers:
            layer.trainable = True

        # change LR
        opt_sgd = tensorflow.keras.optimizers.SGD(
                        lr=1e-3
                        , decay=1e-9
                        , momentum=0.9
                        , nesterov=False
                    )
        new_model.compile(
            loss="categorical_crossentropy"
            , optimizer=opt_sgd
            , metrics=["categorical_accuracy"]
        )
        new_model.load_weights('/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/birds_200_weights.h5')

        new_res = output(imgPath, new_model)
        # print('res[0]', new_res[0])
        # print('res[1]', new_res[1])
    
        New_accu = new_res[0]
        print(" ({:.3})".format(New_accu))
        print('label name:', new_res[1])
        print ('acc after purning (%.2f%%)', New_accu)
        print('difference: ', (orig_acc - New_accu))
        #weights_dic[filt][0] = 1/(k+1)  # resp
        tuple = orig_acc - New_accu, 1
        #weights_dic[filt] = original_loss[1] - loss[1]  # prob
        weights_dic[filt] = tuple
        # arryWeights_last_After[i]=1
        K.clear_session ()

    weights_dic_sort = sorted (weights_dic.items (), key=lambda kv: kv[1], reverse=True)
    print("--- %s seconds ---" % (time.time() - start_time))
    pairKeyValue = { originalName: weights_dic_sort}
    dctOfAllDictionClasses = {}
    dctOfAllDictionClasses.update(pairKeyValue)
    print('pair ley:', pairKeyValue)
    import json
    with open('/content/drive/My Drive/Colab Notebooks/Cub-200_BCNN/Filt_Resp_Birds_BCNN.txt', 'a') as fout:

        fout.write("\n" + str(dctOfAllDictionClasses) + "\n") 
        print('Last Image ID:', imagesFileNames[imgCount])
    imgCount+=1
    indexImg+=1
    
